In [1]:
# # Carregar X e y de arquivos .npy
# X = np.load('X_windows.npy')
# y = np.load('y_labels.npy')
import numpy as np

# Salvar X e y em arquivos .npy
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
X_val = np.load('X_valid.npy')

y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')
y_val = np.load('y_valid.npy')

print("Treino")
print(X_train.shape, y_train.shape)  # Verificar as formas dos dados carregados

print("Teste")
print(X_test.shape, y_test.shape)  # Verificar as formas dos dados carregados


print("Validação")
print(X_val.shape, y_val.shape)  # Verificar as formas dos dados carregados

Treino
(16411, 50, 3, 2) (16411,)
Teste
(4491, 50, 3, 2) (4491,)
Validação
(4491, 50, 3, 2) (4491,)


In [1]:
import h5py

ImportError: DLL load failed while importing _errors: Não foi possível encontrar o procedimento especificado.

In [2]:
num_classes =30  # Certifique-se de que isso corresponde ao número de classes nos seus rótulos.
epochs = 20
window_size = 50

In [2]:
from tensorflow.keras.utils import to_categorical

num_classes = y_train[-1] + 1 
# Realiza one-hot encoding no y
y_train = to_categorical(y_train, num_classes=num_classes)  # considerando que y varia de 0 a 20
y_test = to_categorical(y_test, num_classes=num_classes)  # considerando que y varia de 0 a 20
y_val = to_categorical(y_val, num_classes=num_classes)  # considerando que y varia de 0 a 20

ImportError: cannot import name 'tensorflow' from partially initialized module 'opt_einsum.backends' (most likely due to a circular import) (c:\Users\rodri\anaconda3\envs\py310\lib\site-packages\opt_einsum\backends\__init__.py)

In [3]:
import keras_tuner as kt
from tensorflow import keras
from keras import layers, models
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

ImportError: cannot import name 'tensorflow' from partially initialized module 'opt_einsum.backends' (most likely due to a circular import) (c:\Users\rodri\anaconda3\envs\py310\lib\site-packages\opt_einsum\backends\__init__.py)

In [6]:
# Definir função de construção do modelo com hiperparâmetros
def build_model(hp):
    inputs = layers.Input(shape=(window_size, 3, 2, 1))
    
    # Primeira camada Conv3D com busca de hiperparâmetros para filtros e kernel size
    x = layers.Conv3D(
        filters=hp.Int('filters_1', min_value=32, max_value=128, step=32),
        kernel_size=(3, 3, 2),
        activation='relu',
        padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling3D(pool_size=(2, 1, 1))(x)
    x = layers.Dropout(hp.Float('dropout_1', min_value=0.3, max_value=0.7, step=0.1))(x)

    # Segunda camada Conv3D
    x = layers.Conv3D(
        filters=hp.Int('filters_2', min_value=64, max_value=256, step=64),
        kernel_size=(3, 3, 1),
        activation='relu',
        padding='same',
        kernel_regularizer=l2(hp.Float('l2_reg', min_value=0.001, max_value=0.02, step=0.005)))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling3D(pool_size=(2, 1, 1))(x)

    # Global Average Pooling
    x = layers.GlobalAveragePooling3D()(x)

    # Camada densa para classificação
    output = layers.Dense(num_classes, activation='softmax')(x)

    # Definir o modelo
    model = models.Model(inputs=inputs, outputs=output)

    # Compilar o modelo com taxa de aprendizado como hiperparâmetro
    model.compile(
        optimizer=tf.keras.optimizers.RMSprop(
            learning_rate=hp.Float('learning_rate', min_value=0.0001, max_value=0.01, sampling='log')),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:

# Criar o tuner com RandomSearch
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='cnn_tuning')

# Exibir a pesquisa de hiperparâmetros
tuner.search_space_summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Executar a pesquisa
tuner.search(X_train, y_train, 
             epochs=20, 
             validation_data=(X_val, y_val), 
             callbacks=[early_stopping])

In [ ]:
# Obter o melhor modelo
best_model = tuner.get_best_models(num_models=1)[0]

# Exibir o resumo do melhor modelo
best_model.summary()

# Avaliar o modelo
best_model.evaluate(X_test, y_test)

In [ ]:
import numpy as np
import tensorflow as tf
import json

# Carregar X e y de arquivos .npy
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
X_val = np.load('X_valid.npy')

y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')
y_val = np.load('y_valid.npy')

from tensorflow.keras.utils import to_categorical

# Definir número de classes a partir dos dados
num_classes = y_train.max() + 1

# Realiza one-hot encoding em y
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)

# Achatar os dados de entrada para MLP preservando o janelamento
window_size = 50
X_train_mlp = X_train.reshape((X_train.shape[0], window_size * 3 * 2))  # (50 * 3 * 2) = 300
X_val_mlp = X_val.reshape((X_val.shape[0], window_size * 3 * 2))        
X_test_mlp = X_test.reshape((X_test.shape[0], window_size * 3 * 2))    

import keras_tuner as kt
from tensorflow.keras import layers, models
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Definir função de construção do modelo com hiperparâmetros
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Input(shape=(X_train_mlp.shape[1],)))  # Camada de entrada

    # Primeira camada densa com busca de hiperparâmetros
    model.add(layers.Dense(units=hp.Int('units_1', min_value=128, max_value=512, step=64),
                           activation='relu', kernel_regularizer=l2(hp.Float('l2_reg_1', min_value=0.0001, max_value=0.01, step=0.001))))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(hp.Float('dropout_1', min_value=0.3, max_value=0.7, step=0.1)))

    # Segunda camada densa com busca de hiperparâmetros
    model.add(layers.Dense(units=hp.Int('units_2', min_value=64, max_value=256, step=64),
                           activation='relu', kernel_regularizer=l2(hp.Float('l2_reg_2', min_value=0.0001, max_value=0.01, step=0.001))))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(hp.Float('dropout_2', min_value=0.3, max_value=0.7, step=0.1)))

    # Terceira camada densa
    model.add(layers.Dense(units=hp.Int('units_3', min_value=32, max_value=128, step=32),
                           activation='relu', kernel_regularizer=l2(hp.Float('l2_reg_3', min_value=0.0001, max_value=0.01, step=0.001))))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(hp.Float('dropout_3', min_value=0.3, max_value=0.7, step=0.1)))

    # Camada de saída
    model.add(layers.Dense(num_classes, activation='softmax'))

    # Compilar o modelo
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=0.0001, max_value=0.01, sampling='log')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Criar o tuner com RandomSearch
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=30,
    executions_per_trial=1,
    directory='mlp_tuning',
    project_name='mlp_dnn_tuning')

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Executar a pesquisa
tuner.search(X_train_mlp, y_train, 
             epochs=20, 
             validation_data=(X_val_mlp, y_val), 
             callbacks=[reduce_lr, early_stopping])

# Obter o melhor modelo
best_model = tuner.get_best_models(num_models=1)[0]

# Avaliar o modelo no conjunto de teste
mlp_test_loss, mlp_test_accuracy = best_model.evaluate(X_test_mlp, y_test, verbose=0)

# Obter os melhores hiperparâmetros
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps)

# Salvar os resultados e hiperparâmetros em um arquivo
results = {
    "Test Loss": mlp_test_loss,
    "Test Accuracy": mlp_test_accuracy,
    "Best Hyperparameters": {
        "units_1": best_hps.get('units_1'),
        "dropout_1": best_hps.get('dropout_1'),
        "l2_reg_1": best_hps.get('l2_reg_1'),
        "units_2": best_hps.get('units_2'),
        "dropout_2": best_hps.get('dropout_2'),
        "l2_reg_2": best_hps.get('l2_reg_2'),
        "units_3": best_hps.get('units_3'),
        "dropout_3": best_hps.get('dropout_3'),
        "l2_reg_3": best_hps.get('l2_reg_3'),
        "learning_rate": best_hps.get('learning_rate')
    }
}
print(results)

with open('best_mlp_model_results.json', 'w') as f:
    json.dump(results, f, indent=4)

# Salvar o modelo em um arquivo HDF5
best_model.save('best_mlp_model.h5')

print("Parâmetros e resultados do melhor modelo DNN foram salvos.")


## Lend9o o best model

In [2]:
import h5py

ImportError: DLL load failed while importing _errors: Não foi possível encontrar o procedimento especificado.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Carregar o modelo salvo
#best_model = load_model('best_model.h5')

# Ver o resumo do modelo para garantir que foi carregado corretamente
#best_model.summary()

ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
# Avaliar o modelo no conjunto de teste
test_loss, test_acc = best_model.evaluate(X_test_mlp, y_test, verbose=0)
print(f"Perda (Loss) no conjunto de teste: {test_loss:.4f}")
print(f"Acurácia (Accuracy) no conjunto de teste: {test_acc * 100:.2f}%")

# Fazer previsões em novos dados
predictions = best_model.predict(X_test_mlp)